### READ DATA AND CONVERT INTO CSV
1. setup the wdir to the directory where the grib2 data is collected
2. each directory of the wdir will be opened and the grib inside it processed
    a. each grib file will be converted in a grid, based on DWD weights
    b. 

In [ ]:
wdir = "/home/daniele/documents/github/ftt01/phd/projects/hydrological_forecasting/"

In [ ]:
# IMPORTs
import sys, os
import glob
import subprocess
import pandas as pd
from dask import dataframe as dd

# to link the lib in py scripts as well
os.chdir( wdir )
sys.path.insert( 0, os.path.join(os.path.abspath(os.getcwd()),'lib') )

from lib import *

In [ ]:
# def regrid_dir(path_dir, regridded=False):

#     if regridded == False:

#         os.chdir(path_dir)

#         subprocess.run('''docker run --rm \
#             --volume $(pwd):/local \
#             --env INPUT_FILE=/local \
#             --env OUTPUT_FILE=/local \
#             deutscherwetterdienst/regrid:icon-d2-eps \
#             /convert.sh''',
#                        shell=True, check=True,
#                        executable='/bin/bash')

In [ ]:
# def transform(file_path, destination):

#     copy_process = "cp {} {}".format( file_path, destination + os.path.basename(file_path))
#     subprocess.run(copy_process, shell=True,
#                    check=True, executable='/bin/bash')

#     os.chdir(os.path.dirname(destination))

#     filename = os.path.basename(file_path)

#     transform_command = '''docker run --rm \
#         --volume $(pwd):/local \
#             deutscherwetterdienst/python-eccodes \
#                 grib_copy -B stepRange {filename} temporal_{filename}'''
#     transform_command = transform_command.format(filename=filename)
#     print(transform_command)
#     subprocess.run(transform_command,
#                    shell=True, check=True,
#                    executable='/bin/bash')
    
#     subprocess.run('''rm {}'''.format(filename),
#                    shell=True, check=True,
#                    executable='/bin/bash')


In [ ]:
# SETUP
# data_path = '/mnt/e/projects/hydrological_forecasting/machine_learning/data/forecast/icon-d2-eps_45h/'
# data_path = '/media/windows/projects/hydrological_forecasting/machine_learning/data/forecast/icon-d2-eps_45h/done/'
data_path = '/media/windows/projects/hydro_forecasting/machine_learning/data/forecast/icon-d2-eps_45h/todo/'

# dummy_output = wdir + 'data/passirio/dummy_output.csv'

variables = ['t_2m','tot_prec']
init_ref = '03'
lead_hours = 45
ensemble_number = 20

## Passirio basin
basin = 'passirio'

# ## Plan basin
# basin = 'plan'

In [ ]:
dirs = glob.glob( data_path + '*/' )
dirs

In [ ]:
# # Include standard modules
# import argparse

# # Initiate the parser
# parser = argparse.ArgumentParser()

# # Add long and short argument
# parser.add_argument("--path", "-p", help="set input path")

# # Read arguments from the command line
# args = parser.parse_args()

# # Check for --path
# if args.path:
#     print("Set path to %s" % args.path)
#     dirs = [args.path]

In [ ]:
lead_time_array = []
for n in range(1, lead_hours+1):
    n = str(n).zfill(3)
    lead_time_array.append(n)

for el in dirs:

    dir_gribs = glob.glob(el + '*.grib2')

    if len(dir_gribs) != 0:
        for dir_grib in dir_gribs:
            rm_process = "rm {}".format(dir_grib)
            print(rm_process)
            subprocess.run(rm_process, shell=True,
                           check=True, executable='/bin/bash')

    # print(el)
    tmp = el.split('/')
    current_date = str(tmp[len(tmp) - 2])

    # print("Current date: " + current_date)

    for variable in variables:

        # print("Variable: " + variable)

        #current_file = []
        for n in range(1, lead_hours+1):

            n = str(n).zfill(3)
            # print("Lead hour: " + n)

            # create inner dirs structure
            current_file_path = el + n + '/' + basin + '/' + variable + '/'
            # print("Current dir: " + current_file_path)

            original_current_file = glob.glob(current_file_path)

            if len(original_current_file) != 0:
                gribs = glob.glob(original_current_file[0] + '*.grib2')

                if len(gribs) != 0:
                    for grib in gribs:

                        rm_process = "rm {}".format(grib)
                        print(rm_process)
                        subprocess.run(rm_process, shell=True,
                                       check=True, executable='/bin/bash')
                # else:
                #     print('Missing GRIB2 files' + current_file_path)

                output_completes = glob.glob(
                    original_current_file[0] + 'output.csv')

                if len(output_completes) != 0:
                    for output_complete in output_completes:

                        rm_process = "rm {}".format(output_complete)
                        print(rm_process)
                        subprocess.run(rm_process, shell=True,
                                       check=True, executable='/bin/bash')
                # else:
                    # print('Missing OUTPUT files' + current_file_path)

            # else:
            #     print('Missing DIR')


In [ ]:
ensembles = range(1, ensemble_number + 1)

In [ ]:
# consistency check

for el in dirs:

    tmp = el.split('/')
    current_date = str(tmp[len(tmp) - 2])

    for variable in variables:

        for n in range(1, lead_hours+1):

            n = str(n).zfill(3)

            for ensemble in ensembles:

                ensemble = str(ensemble).zfill(3)

                file_to_check = el + n + '/' + basin + '/' + variable + '/' + ensemble + '/' + 'output.csv'

                if not( os.path.isfile(file_to_check) ):
                    print("Current date: " + current_date)
                    print("Variable: " + variable)
                    print("MISSING: " + file_to_check)